# Diabetes Risk Prediction from Lifestyle Factors

End‑to‑end: EDA, CV models, threshold tuning, calibration, SHAP.

In [ ]:

import os, json, joblib, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, average_precision_score, confusion_matrix,
                             roc_curve, precision_recall_curve, brier_score_loss)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import shap, numpy as _np
if not hasattr(_np, "bool"): _np.bool = _np.bool_
warnings.filterwarnings("ignore"); np.random.seed(42)
DATA_PATH = Path("data/cleaned_diabetes_lifestyle.csv")


## 1. Load Data

In [ ]:

df = pd.read_csv(DATA_PATH).dropna(subset=["Diabetic"]).copy()
print("Shape:", df.shape); df.head()


## 2. EDA

In [ ]:

counts = df['Diabetic'].value_counts().sort_index()
plt.figure(); plt.bar(['Non-diabetic(0)','Diabetic(1)'], counts.values); plt.title('Class Distribution'); plt.show()
num_cols = [c for c in ['Age','BMI','Sleep','SoundSleep','Pregancies'] if c in df.columns]
df[num_cols].describe()


In [ ]:

for col in [c for c in ['BMI','Sleep','SoundSleep'] if c in df.columns]:
    plt.figure(); plt.boxplot([df[df['Diabetic']==0][col], df[df['Diabetic']==1][col]], labels=['0','1'])
    plt.title(f'{col} by Diabetes'); plt.ylabel(col); plt.show()


## 3. Preprocessing

In [ ]:

X = df.drop(columns=['Diabetic']); y = df['Diabetic'].astype(int)
numeric_cols = [c for c in ['Age','BMI','Sleep','SoundSleep','Pregancies'] if c in X.columns]
other_cols = [c for c in X.columns if c not in numeric_cols]
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
other_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent'))])
preprocess = ColumnTransformer([('num', numeric_transformer, numeric_cols), ('other', other_transformer, other_cols)], remainder='drop')


## 4. CV Model Comparison

In [ ]:

models = {
    "LogisticRegression": LogisticRegression(max_iter=2000, class_weight="balanced", solver="lbfgs"),
    "RandomForest": RandomForestClassifier(n_estimators=200, min_samples_leaf=2, class_weight="balanced", random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(kernel="rbf", probability=True, class_weight="balanced", random_state=42)
}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"roc_auc":"roc_auc","avg_prec":"average_precision","f1":"f1","recall":"recall","precision":"precision","accuracy":"accuracy"}
rows=[]
for name, model in models.items():
    pipe = Pipeline([("preprocess", preprocess), ("model", model)])
    cv = cross_validate(pipe, X, y, cv=skf, scoring=scoring)
    rows.append({"Model":name,"ROC_AUC (CV mean)":float(np.mean(cv["test_roc_auc"])),
                 "PR_AUC (CV mean)":float(np.mean(cv["test_avg_prec"])),
                 "F1 (CV mean)":float(np.mean(cv["test_f1"])),
                 "Recall (CV mean)":float(np.mean(cv["test_recall"])),
                 "Precision (CV mean)":float(np.mean(cv["test_precision"])),
                 "Accuracy (CV mean)":float(np.mean(cv["test_accuracy"]))})
cv_df = pd.DataFrame(rows).sort_values("ROC_AUC (CV mean)", ascending=False).reset_index(drop=True)
cv_df


## 5. Train Best, Tune Threshold, Curves

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)
best_name = cv_df.iloc[0]["Model"]; best_model = models[best_name]
best_pipe = Pipeline([("preprocess", preprocess), ("model", best_model)]).fit(X_train, y_train)
proba = best_pipe.predict_proba(X_test)[:,1]; pred = (proba>=0.5).astype(int)
def metrics(y_true, y_prob, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return {"Accuracy":accuracy_score(y_true,y_pred),"Precision":precision_score(y_true,y_pred),
            "Recall":recall_score(y_true,y_pred),"F1":f1_score(y_true,y_pred),
            "ROC_AUC":roc_auc_score(y_true,y_prob),"PR_AUC":average_precision_score(y_true,y_prob),
            "TN":int(tn),"FP":int(fp),"FN":int(fn),"TP":int(tp)}
default_metrics = metrics(y_test, proba, pred); pd.DataFrame([default_metrics])


In [ ]:

ths = np.linspace(0.01,0.99,99); f1s=[f1_score(y_test,(proba>=t).astype(int)) for t in ths]
best_t = float(ths[int(np.argmax(f1s))]); pred_t = (proba>=best_t).astype(int)
tuned_metrics = metrics(y_test, proba, pred_t); tuned_metrics["Threshold"]=best_t
pd.DataFrame([tuned_metrics])


In [ ]:

fpr, tpr, _ = roc_curve(y_test, proba)
plt.figure(); plt.plot(fpr,tpr,label='ROC'); plt.plot([0,1],[0,1],'--'); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {best_name}'); plt.legend(); plt.show()
prec, rec, _ = precision_recall_curve(y_test, proba)
plt.figure(); plt.plot(rec,prec,label='PR'); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title(f'PR — {best_name}'); plt.legend(); plt.show()


## 6. Calibration (Platt)

In [ ]:

platt = CalibratedClassifierCV(RandomForestClassifier(n_estimators=200, min_samples_leaf=2, class_weight='balanced', random_state=42),
                               method='sigmoid', cv=3)
cal_pipe = Pipeline([("preprocess", preprocess), ("model", platt)]).fit(X_train, y_train)
proba_cal = cal_pipe.predict_proba(X_test)[:,1]; pred_cal = (proba_cal>=0.5).astype(int)
cal_metrics = metrics(y_test, proba_cal, pred_cal); cal_metrics["Brier"]=brier_score_loss(y_test, proba_cal)
pd.DataFrame([cal_metrics])


## 7. SHAP (global, local, interactions)

In [ ]:

if best_name in ["RandomForest","GradientBoosting"]:
    feats = best_pipe.named_steps["preprocess"].get_feature_names_out()
    Xt = best_pipe.named_steps["preprocess"].transform(X_test)
    model = best_pipe.named_steps["model"]
    expl = shap.TreeExplainer(model)
    vals = expl.shap_values(Xt)
    shap.summary_plot(vals[1], Xt, feature_names=feats, plot_type="bar")
    shap.summary_plot(vals[1], Xt, feature_names=feats)
    pos = int(np.where(y_test.values==1)[0][0]); neg = int(np.where(y_test.values==0)[0][0])
    shap.plots._waterfall.waterfall_legacy(expl.expected_value[1], vals[1][pos,:], feature_names=feats, max_display=20)
    shap.plots._waterfall.waterfall_legacy(expl.expected_value[1], vals[1][neg,:], feature_names=feats, max_display=20)
    sample = min(150, Xt.shape[0]); inter = expl.shap_interaction_values(Xt[:sample])
    inter_mean = np.abs(inter[1]).mean(axis=0)
    import pandas as pd, matplotlib.pyplot as plt
    inter_df = pd.DataFrame(inter_mean, index=feats, columns=feats)
    top = inter_df.mean(axis=1).sort_values(ascending=False).head(15).index
    plt.figure(figsize=(8,6)); plt.imshow(inter_df.loc[top, top], aspect='auto'); plt.colorbar(); plt.title('SHAP Interaction Heatmap (Top 15)')
    plt.xticks(range(len(top)), top, rotation=90); plt.yticks(range(len(top)), top); plt.tight_layout(); plt.show()
else:
    print("Best model not tree-based; SHAP skipped.")


## 8. Save Pipeline & Metadata

In [ ]:

import joblib, json
from pathlib import Path
Path("models").mkdir(exist_ok=True, parents=True)
joblib.dump(best_pipe, Path("models")/"diabetes_lifestyle_pipeline.pkl")
meta = {"best_model":best_name,"threshold_default":0.5,"threshold_max_f1":best_t,
        "default_metrics":default_metrics,"tuned_metrics":tuned_metrics}
with open(Path("models")/"diabetes_lifestyle_pipeline_info.json","w") as f: json.dump(meta, f, indent=2)
print("Saved to models/")
